### instantiating services

In [ ]:
from apps.homebase.entities import ProposalStatus, Proposal, StateInContract, Txaction, Token, Member, Org, Vote
from apps.homebase.abis import wrapperAbi, daoAbiGlobal,tokenAbiGlobal
from datetime import datetime
import time
from firebase_admin import initialize_app
from firebase_admin import firestore,credentials
from web3 import Web3
import re
cred = credentials.Certificate('homebase.json')
# initialize_app(cred)
db = firestore.client()
networks=db.collection("contracts")
ceva=networks.document("Etherlink-Testnet").get()
wrapper_address = ceva.to_dict()['wrapper']
print("wrapper address :" + str(wrapper_address))
rpc="https://node.ghostnet.etherlink.com"
web3 = Web3(Web3.HTTPProvider(rpc))
papers={}
daos=[]
if web3.is_connected():
    print("node connected")
else:
    print("node connection failed!")

wabi=re.sub(r'\n+', ' ', wrapperAbi).strip()
tabi=re.sub(r'\n+', ' ', tokenAbiGlobal).strip()
wrapper_contract = web3.eth.contract(address=wrapper_address, abi=wabi)
daos_collection=db.collection('idaosEtherlink-Testnet')
docs = list(daos_collection.stream())
dao_addresses = [doc.id for doc in docs]

class Paper:
    ZERO_ADDRESS = "0x0000000000000000000000000000000000000000"
    def __init__(self, address,kind,dao=None):
        self.address = address
        self.kind=kind
        self.contract=None
        self.dao=dao
        if kind=="wrapper":
            self.abi=re.sub(r'\n+', ' ', wrapperAbi).strip()
        elif kind == "token":
            self.abi = re.sub(r'\n+', ' ', tokenAbiGlobal).strip()
        else:
            self.abi = None

    def get_contract(self):
        if self.contract==None:
            self.contract=web3.eth.contract(address=self.address, abi=self.abi) 
        return self.contract
    
    def add_dao(self,log):
        decoded_event = wrapper_contract.events.NewDaoCreated().process_log(log)
        name = decoded_event['args']['name']
        print("new dao detected: "+name)
        org:Org=Org(name=name)
        org.creationDate=datetime.now()
        org.govTokenAddress=decoded_event['args']['token']
        org.address = decoded_event['args']['dao']
        org.symbol = decoded_event['args']['symbol']
        org.registryAddress=decoded_event['args']['registry']
        org.description = decoded_event['args']['description']
        members=decoded_event['args']['initialMembers']
        amounts=decoded_event['args']['initialAmounts']
        org.holders=len(members)
        supply=0
        batch = db.batch()
        for num in range(len(members)):
            m:Member=Member(address=members[num],personalBalance=amounts[num],delegate="",votingWeight="0")
            member_doc_ref = daos_collection \
                    .document(org.address) \
                    .collection('members') \
                    .document(m.address)
            batch.set(reference=member_doc_ref, document_data=m.toJson())
            supply=supply+amounts[num]
        org.totalSupply=str(supply)
        keys=decoded_event['args']['keys']
        values=decoded_event['args']['values']
        org.registry={keys[i]: values[i] for i in range(len(keys))}
        org.quorum=decoded_event['args']['initialAmounts'][-1]
        org.proposalThreshold=decoded_event['args']['initialAmounts'][-2]
        org.votingDuration=decoded_event['args']['initialAmounts'][-3]
        org.treasuryAddress="0xFdEe849bA09bFE39aF1973F68bA8A1E1dE79DBF9"
        org.votingDelay=decoded_event['args']['initialAmounts'][-4]
        org.executionDelay=decoded_event['args']['executionDelay']
        token_contract = web3.eth.contract(address=org.govTokenAddress, abi=tabi)
        org.decimals = token_contract.functions.decimals().call()
        daos_collection.document(org.address).set(org.toJson())
        batch.commit()
        return org.address
    
    def delegate(self,log):
        contract=self.get_contract()
        data=contract.events.DelegateChanged().process_log(log)
        delegator= data['args']['delegator']
        fromDelegate=data['args']['fromDelegate']
        toDelegate=data['args']['toDelegate']
        batch = db.batch()
        delegator_doc_ref = daos_collection \
                    .document(self.dao) \
                    .collection('members') \
                    .document(delegator)
        batch.set(reference=delegator_doc_ref, document_data={"delegate":toDelegate})
        if delegator!=toDelegate:
            print("delegating to someone else")
            toDelegate_doc_ref = daos_collection \
                    .document(self.dao) \
                    .collection('members') \
                    .document(toDelegate).collection("constituents").document(delegator)
            batch.set(reference=toDelegate_doc_ref, document_data={"address":delegator})

            if fromDelegate and fromDelegate != self.ZERO_ADDRESS and fromDelegate != delegator:
                    fromDelegate_doc_ref = daos_collection \
                        .document(self.dao) \
                        .collection('members') \
                        .document(fromDelegate) \
                        .collection("constituents") \
                        .document(delegator)
                    batch.delete(fromDelegate_doc_ref)
        batch.commit()
        return None

    def handle_event(self,log):
        if self.kind=="wrapper":
            self.add_dao(log)
        if self.kind=="token":
            self.delegate(log)

#I hate my life

for doc in docs:
    obj=doc.to_dict()
    try:
        p=Paper(address=obj['token'],kind="token",dao=doc.id)
    except Exception as e:
        print("one DAO contract can't parse correctly: "+str(e))
    papers.update({obj['token']:p})

event_signatures = {
    web3.keccak(text="NewDaoCreated(address,address,address[],uint256[],string,string,string,uint256,address,string[],string[])").hex(): "NewDaoCreated",
    "0x3134e8a2e6d97e929a7e54011ea5485d7d196dd5f0ba4d4ef95803e8e3fc257f": "DelegateChanged",
}

papers.update({wrapper_address:Paper(wrapper_address,"wrapper")})
listening_to_addresses=[wrapper_address]
listening_to_addresses=listening_to_addresses+list(papers.keys())


counter=0
processed_transactions = set()
heartbeat=0
print(f"Listening for {len(event_signatures)} events on {len(papers.items())} contracts...")

while True:
    heartbeat+=1
    try:
        latest=web3.eth.block_number
        first=latest-4
        logs = web3.eth.get_logs({
            "fromBlock": first,
            "toBlock": latest,
            "address": listening_to_addresses,
            "topics": [[*event_signatures.keys()], None]
        })
        for log in logs:
            tx_hash = log["transactionHash"].hex()
            if tx_hash in processed_transactions:
                print("already did this one")
                continue  # Skip duplicate
            contract_address = log["address"]
            event_signature = log["topics"][0].hex()
            print("we caught event in address" + str(contract_address))
            event_name = event_signatures.get(event_signature)
            print(f"Event: {event_name}, Contract: {contract_address}")
            new_contract_addresses=papers[contract_address].handle_event(log)
            if new_contract_addresses != None:
                dao_address=new_contract_addresses[0]
                token_address=new_contract_addresses[1]
                listening_to_addresses=listening_to_addresses+[dao_address]+[token_address]
                papers.update({token_address:Paper(address=token_address,kind="token", dao=dao_address )})
            processed_transactions.add(tx_hash)
    except Exception as e:
        print("something went wrong "+str(e))

    if heartbeat%50==0:
        print("heartbeat: "+str(heartbeat))

    time.sleep(3)
    

wrapper address :0xDF38eD7d9DcF15577F0E6eb619b945669F1f74b1
node connected
Listening for 2 events on 10 contracts...
we caught event in address0xDF38eD7d9DcF15577F0E6eb619b945669F1f74b1
Event: NewDaoCreated, Contract: 0xDF38eD7d9DcF15577F0E6eb619b945669F1f74b1
new dao detected: Monday
heartbeat: 50


KeyboardInterrupt: 

In [7]:
logs

[]

In [9]:
a=[1,2,3]
b=[4]
c=[6]
d=a+b+c
d

[1, 2, 3, 4, 6]

In [10]:
logs=[
  {
    "address": "0xcfdec38da296e439a0a865519e8b48a5b0bff5b4",
    "topics": [
      "0x3134e8a2e6d97e929a7e54011ea5485d7d196dd5f0ba4d4ef95803e8e3fc257f",
      "0x000000000000000000000000a9f8f9c0bf3188ceddb9684ae28655187552bae9",
      "0x000000000000000000000000a9f8f9c0bf3188ceddb9684ae28655187552bae9",
      "0x000000000000000000000000a9f8f9c0bf3188ceddb9684ae28655187552bae9"
    ],
    "data": "0x",
    "blockNumber": "0xd9936d",
    "transactionHash": "0xb4535077457b25e15a943c6cfb18fd48f2d3de3f53b62deb6c8ac5bac8689ff1",
    "transactionIndex": "0x0",
    "blockHash": "0x9147729e6f852ca605084f3ddf215295dbfa1c9b1a28561427408108121a7ee5",
    "logIndex": "0x0",
    "removed": False
  }
]

In [11]:
contract:web3.eth.contract=papers['0xa0e858BB3dD3eF3C0B08eE563c6c2a46820C88A3'].get_contract()
ceva=contract.events.DelegateChanged().process_log(logs[0])

In [12]:
ceva['args']

{'delegator': '0xa9F8F9C0bf3188cEDdb9684ae28655187552bAE9',
 'fromDelegate': '0xa9F8F9C0bf3188cEDdb9684ae28655187552bAE9',
 'toDelegate': '0xa9F8F9C0bf3188cEDdb9684ae28655187552bAE9'}

In [ ]:
decoded = papers['0xa0e858BB3dD3eF3C0B08eE563c6c2a46820C88A3'].get_contract().events.DelegateVotesChanged().process_log(log)

In [4]:
tabi=re.sub(r'\n+', ' ', tokenAbiGlobal).strip()
token_contract = web3.eth.contract(address="0xFdEe849bA09bFE39aF1973F68bA8A1E1dE79DBF9", abi=tabi)


# Call the read-only function


print(f"Address value: {address_variable_value}")


Address value: 2


In [8]:
type(address_variable_value)

int

In [23]:
first=web3.eth.block_number-4
logs = web3.eth.get_logs({
    "fromBlock": first,
    "toBlock": web3.eth.block_number,
    "address": wrapper_address,
    "topics": [
        web3.keccak(text="NewDaoCreated(address,address,address[],uint256[],string,string,string,uint256)").hex(),
        None,  # Match any dao address
    ]
})
len(logs)

0

In [24]:
first

13872535

In [9]:
decoded_event = wrapper_contract.events.NewDaoCreated().process_log(logs[0])

In [10]:
decoded_event

AttributeDict({'args': AttributeDict({'dao': '0x833365797370B3fE7F5B86E10FA62195e74b102e',
  'token': '0x69d7aF22AdDECA38b47686237c5a3d6Ce915111E',
  'initialMembers': ['0x36fE1912506E4b3B85e5a4e102237Ea81da1C3d3'],
  'initialAmounts': [100000, 1, 1, 1, 1],
  'name': 'aced',
  'symbol': 'ADS',
  'description': 'this is the description',
  'executionDelay': 90}),
 'event': 'NewDaoCreated',
 'logIndex': 11,
 'transactionIndex': 1,
 'transactionHash': HexBytes('0x36e1197c733317bed02a9cf6e9bb084ba8ccc5a04cdff9072c51a7fd9c46530c'),
 'address': '0xA3C99359F376bE32ADE1C259Faa01e119B038436',
 'blockHash': HexBytes('0x9570374a8a889a9c65db2b6c54d313caebfbbbac6ec02c14ad45089199c5f5f8'),
 'blockNumber': 13871169})

In [20]:
project_addresses = [doc.id for doc in projects_collection.stream()]
project_addresses

['0x379080EE8941CE5Ae5560d5E32A4CEBBaC7C6b92']

In [10]:
a=[1,2,3,4,5,6,7,8,9]
a[-2]

8

In [26]:

def listen_to_project_events(contract, contract_address):
    global processed_transactions
    
    # Define the topics for all events
    event_signatures = {
        "SetParties": web3.keccak(text="SetParties(address,address,string)").hex(),
        "SendFunds": web3.keccak(text="SendFunds(address,uint256)").hex(),
        "ContractorPaid": web3.keccak(text="ContractorPaid(address,uint256)").hex(),
        "ContributorWithdrawn": web3.keccak(text="ContributorWithdrawn(address,uint256)").hex(),
        "ProjectDisputed": web3.keccak(text="ProjectDisputed(address)").hex(),
        "ProjectClosed": web3.keccak(text="ProjectClosed(address)").hex(),
        "ContractSigned": web3.keccak(text="ContractSigned(address)").hex(),
        "ArbitrationDecision": web3.keccak(text="ArbitrationDecision(address,uint256,string)").hex(),
    }

    # Get logs for the current contract
    logs = web3.eth.get_logs({
        "fromBlock": web3.eth.block_number-3,
        "toBlock": web3.eth.block_number,
        "address": contract_address,
    })

    for log in logs:
        tx_hash = log["transactionHash"].hex()
        if tx_hash in processed_transactions:
            print("already did this one")
            continue  # Skip already processed transactions

        try:
            event_type = None
            for event_name, topic_hash in event_signatures.items():
                print("got one!")
                if log["topics"][0].hex() == topic_hash:
                    event_type = event_name
                    break

            if not event_type:
                print("Not an avent type")
                continue  # Skip logs that don't match any defined events

            # Decode and process the event
            decoded_event = getattr(contract.events, event_type)().process_log(log)
            event_data = getattr(ProjectEvents, event_type)(decoded_event)

            print(f"Event detected in contract {contract_address} ({event_type}):", event_data)

        except Exception as e:
            print(f"Error processing log for contract {contract_address}: {e}")

        # Mark the transaction as processed
        processed_transactions.add(tx_hash)


In [ ]:
while True:
    heartbeat += 1

    for address in project_addresses:
        # Get contract instance
        contract = web3.eth.contract(address=address, abi=native_project_abi)
        try:
            listen_to_project_events(contract, address)
        except Exception as e:
            
            print("Errrorrr "+e)

    if heartbeat % 10 == 0:
        print(f"Heartbeat: {heartbeat}")

    time.sleep(3)

In [ ]:
import apps.services

In [ ]:
import requests
dorg_homebase_channel_id = "804689936388325376"

def send_discord_message(msg, channel_id):
    global discord_bot_token
    url = "https://discordapp.com/api/channels/"+channel_id+"/messages"
    headers = {
        "Authorization": "Bot " + discord_bot_token
    }
    body = {
        "content": msg
    }
    print("before sending "+msg)
    response = requests.request("POST", url, headers=headers, data=body)
    return {"data": str("Status code "+str(response.status_code))}

In [31]:
send_discord_message(channel_id=dorg_homebase_channel_id, msg="INDEXER IS DOWN ON TRUSTLESS!")

before sending INDEXER IS DOWN ON TRUSTLESS!


{'data': 'Status code 200'}

In [33]:
from dotenv import dotenv_values
env_variables = dotenv_values(".env")

In [ ]:
my_variable = env_variables["discord_bot_token"]
print(my_variable)